In [1]:
# Import libraries and read in datasets
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings('ignore')
df = pd.read_csv('reviews.tsv', sep='\t', on_bad_lines='skip')


In [2]:
df.head()

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,53096384,R63J84G1LOX6R,1563890119,763187671,The Sandman Vol. 1: Preludes and Nocturnes,Books,4.0,0.0,1.0,N,N,ignore the review below,this is the first 8 issues of the series. it i...,1995-08-13
1,US,53096399,R1BALOA11Z06MT,1559947608,381720534,The 22 Immutable Laws of Marketing,Books,4.0,0.0,0.0,N,N,awesome,I've always been partial to immutable laws. Th...,1995-08-17
2,US,53096332,R1LLAY5W5PZUS4,0671701800,860650224,Contact,Books,5.0,1.0,2.0,N,N,Read the book. It's good.,This is a book about first contact with aliens...,1995-08-30
3,US,53096335,R3R9VTJ82FXECQ,0425132153,624269601,Good Omens,Books,5.0,0.0,0.0,N,N,Funniest book ever written about the Anti-Christ,This is quite possibly *the* funniest book I h...,1995-09-11
4,US,51747709,R1P5J3FNBWTFXY,0517122707,161411385,A Confederacy of Dunces,Books,5.0,0.0,0.0,N,N,A winner that didn't last. Only a mothers love...,The story behind the book is almost better tha...,1995-10-17


In [3]:
# Find out how many downvotes each review received
df['downvotes'] = df.total_votes - df.helpful_votes

# Reorder columns
columns = ['marketplace', 'customer_id', 'review_id', 'product_id',
       'product_parent', 'product_title', 'product_category', 'star_rating',
       'helpful_votes', 'downvotes', 'total_votes', 'vine', 'verified_purchase',
       'review_headline', 'review_body', 'review_date']

df = df[columns]


In [4]:
# Standardized product category names
df.product_category.replace({
        'Digital_Music_Purchase': 'Digital Music Purchase',
        'Digital_Ebook_Purchase': 'Digital eBook Purchase',
        'Digital_Video_Download': 'Digital Video Download',
        'Lawn and Garden': 'Lawn & Garden',
        'Mobile_Electronics': 'Mobile Electronics',
        'Personal_Care_Appliances': 'Personal Care Appliances',
        'Mobile_Apps': 'Mobile Apps'
    }, inplace=True)

df.product_category.unique()

array(['Books', 'Music', 'Video', 'Video DVD', 'Toys', 'Tools',
       'Office Products', 'Video Games', 'Software',
       'Digital Music Purchase', 'Home Entertainment', 'Electronics',
       'Digital eBook Purchase', 'Digital Video Download', 'Kitchen',
       'Camera', 'Outdoors', 'Musical Instruments', 'Sports', 'Watches',
       'PC', 'Home', 'Wireless', 'Beauty', 'Baby', 'Home Improvement',
       'Apparel', 'Shoes', 'Lawn & Garden', 'Mobile Electronics',
       'Health & Personal Care', 'Grocery', 'Luggage',
       'Personal Care Appliances', 'Automotive', 'Mobile Apps',
       'Furniture', '2012-12-22', 'Pet Products'], dtype=object)

In [5]:
allowed_categories = ["Sports", "Musical Instruments", "Shoes", "Electronics", "Watches", "Health & Personal Care", "Tools"]

# Filter the DataFrame based on the product_category column
df = df[df['product_category'].isin(allowed_categories)]

# Reset index if needed
df.reset_index(drop=True, inplace=True)

In [6]:
print('Number of reviews: {:,}'.format(len(df)))
print('Number of customers: {:,}'.format(len(df.customer_id.unique())))
print('Number of product parents: {:,}'.format(len(df.product_parent.unique())))
print('Number of product titles: {:,}'.format(len(df.product_title.unique())))
print('Number of unique products: {:,}'.format(len(df.product_id.unique())))
print('Number of product categories: {:,}'.format(len(df.product_category.unique())))
print('Number of days a review was made on: {:,}'.format(len(df.review_date.unique())))

Number of reviews: 52,878
Number of customers: 51,534
Number of product parents: 388
Number of product titles: 526
Number of unique products: 1,560
Number of product categories: 7
Number of days a review was made on: 2,899


In [7]:
# Many lost unique product ids were actually small variations of the same product
df.groupby('product_parent')['product_title'].value_counts()[12:20]

product_parent  product_title                                                                         
29921795        Dragon Bruce Lee Game of Death yellow yellow rubber nunchaku (japan import)                10
30572331        Up                                                                                         11
34053686        Casio A178WGA-1A Mens Gold Tone Stainless Steel Digital Watch Chronograph A178WG            2
39213084        Swatch Men's YCS410GX Windfall Chronograph Silver-Tone Bracelet Watch                      73
                Swatch Irony Chronograph Windfall Mens Watch YCS410GX                                       1
42685970        Sennheiser Professional Headphones with Sound Channeling                                    1
44258853        Behringer Xenyx 802 Premium 8-Input 2-Bus Mixer with Xenyx Mic Preamps and British EQs    435
                Behringer XENYX 802 Mixer                                                                   2
Name: count, dtyp

In [8]:

# Summary statistics of original dataset
df.product_id.value_counts().describe()

count    1560.000000
mean       33.896154
std       160.987713
min         1.000000
25%         1.000000
50%         3.000000
75%        11.000000
max      3970.000000
Name: count, dtype: float64

In [9]:
# Extract counts on how many customers purchased a unique product
purchased_counts = df.product_id.value_counts().rename('purchased_counts')

# Add column containing product purchased counts
df = df.merge(purchased_counts.to_frame(),
                left_on='product_id',
                right_index=True)

# Filter only products with 100 or more purchases
df = df[df.purchased_counts >= 100]

In [10]:
print('Number of reviews: {:,}'.format(len(df)))
print('Number of customers: {:,}'.format(len(df.customer_id.unique())))
print('Number of product parents: {:,}'.format(len(df.product_parent.unique())))
print('Number of product titles: {:,}'.format(len(df.product_title.unique())))
print('Number of unique products: {:,}'.format(len(df.product_id.unique())))
print('Number of product categories: {:,}'.format(len(df.product_category.unique())))
print('Number of days a review was made on: {:,}'.format(len(df.review_date.unique())))

Number of reviews: 39,277
Number of customers: 38,364
Number of product parents: 95
Number of product titles: 97
Number of unique products: 98
Number of product categories: 7
Number of days a review was made on: 2,738


In [11]:
# New summary statistics
df.product_id.value_counts().describe()

count      98.000000
mean      400.785714
std       516.897411
min       102.000000
25%       134.750000
50%       211.000000
75%       476.750000
max      3970.000000
Name: count, dtype: float64

In [12]:
# Number of reviews in each product category
df.groupby('product_category')['product_category'].count()

product_category
Electronics               9290
Health & Personal Care     971
Musical Instruments       9473
Shoes                      281
Sports                    3483
Tools                     7023
Watches                   8756
Name: product_category, dtype: int64

In [13]:
df.groupby('product_parent')['product_id'].value_counts().head()

product_parent  product_id
1348996         B0006AAS5G    551
29559888        B0015HOFZI    353
44258853        B000J5XS3C    435
49980879        B001QWBM62    616
64876957        B00AH3DCAA    137
Name: count, dtype: int64

In [14]:
df.groupby('product_parent')['product_title'].value_counts().head()

product_parent  product_title                                                                              
1348996         Invicta Men's 9211 Speedway Collection Stainless Steel Chronograph Watch with Link Bracelet    551
29559888        Sony PSLX300USB USB Stereo Turntable                                                           353
44258853        Behringer Xenyx 802 Premium 8-Input 2-Bus Mixer with Xenyx Mic Preamps and British EQs         435
49980879        Zoom H4N Handy Portable Digital Recorder - 2009 Version                                        616
64876957        Invicta Men's 9937 Pro Diver Collection Coin-Edge Swiss Automatic Watch                        137
Name: count, dtype: int64

In [15]:
df.groupby('product_parent')['product_category'].value_counts().head()

product_parent  product_category   
1348996         Watches                551
29559888        Electronics            353
44258853        Musical Instruments    435
49980879        Musical Instruments    616
64876957        Watches                137
Name: count, dtype: int64

In [16]:
example = df[df.product_parent == 44258853]
example.groupby('product_parent')['product_title'].value_counts()

product_parent  product_title                                                                         
44258853        Behringer Xenyx 802 Premium 8-Input 2-Bus Mixer with Xenyx Mic Preamps and British EQs    435
Name: count, dtype: int64

In [17]:
def missing(dataset):
    columns = dataset.columns
    print('MISSING ROWS per COLUMN')
    for column in columns:
        percentage = (dataset[column].isnull().sum() / len(dataset)) * 100
        print('{}: {}, {:0.2f}%'.format(column, dataset[column].isnull().sum(), percentage))
        
missing(df)

MISSING ROWS per COLUMN
marketplace: 0, 0.00%
customer_id: 0, 0.00%
review_id: 0, 0.00%
product_id: 0, 0.00%
product_parent: 0, 0.00%
product_title: 0, 0.00%
product_category: 0, 0.00%
star_rating: 0, 0.00%
helpful_votes: 0, 0.00%
downvotes: 0, 0.00%
total_votes: 0, 0.00%
vine: 0, 0.00%
verified_purchase: 0, 0.00%
review_headline: 0, 0.00%
review_body: 4, 0.01%
review_date: 0, 0.00%
purchased_counts: 0, 0.00%


In [18]:
df.dropna(axis=0, inplace=True)
missing(df)

MISSING ROWS per COLUMN
marketplace: 0, 0.00%
customer_id: 0, 0.00%
review_id: 0, 0.00%
product_id: 0, 0.00%
product_parent: 0, 0.00%
product_title: 0, 0.00%
product_category: 0, 0.00%
star_rating: 0, 0.00%
helpful_votes: 0, 0.00%
downvotes: 0, 0.00%
total_votes: 0, 0.00%
vine: 0, 0.00%
verified_purchase: 0, 0.00%
review_headline: 0, 0.00%
review_body: 0, 0.00%
review_date: 0, 0.00%
purchased_counts: 0, 0.00%


In [19]:
# Reset index and drop unimportant columns
df = df.reset_index().drop(['index', 'marketplace', 'vine'], axis=1)

In [20]:
df['verified_purchase'].value_counts()

verified_purchase
Y    34631
N     4642
Name: count, dtype: int64

In [21]:
df.groupby('star_rating')['star_rating'].count()

star_rating
1.0     2935
2.0     1928
3.0     2929
4.0     7437
5.0    24044
Name: star_rating, dtype: int64

In [22]:

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39273 entries, 0 to 39272
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   customer_id        39273 non-null  int64  
 1   review_id          39273 non-null  object 
 2   product_id         39273 non-null  object 
 3   product_parent     39273 non-null  int64  
 4   product_title      39273 non-null  object 
 5   product_category   39273 non-null  object 
 6   star_rating        39273 non-null  float64
 7   helpful_votes      39273 non-null  float64
 8   downvotes          39273 non-null  float64
 9   total_votes        39273 non-null  float64
 10  verified_purchase  39273 non-null  object 
 11  review_headline    39273 non-null  object 
 12  review_body        39273 non-null  object 
 13  review_date        39273 non-null  object 
 14  purchased_counts   39273 non-null  int64  
dtypes: float64(4), int64(3), object(8)
memory usage: 4.5+ MB


In [23]:
df.head()

,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,downvotes,total_votes,verified_purchase,review_headline,review_body,review_date,purchased_counts
0,51020657,RYVGOUKMZQ5NS,B00002N9ER,805377466,Maglite Black Universal Mounting Brackets for ...,Tools,5.0,2.0,2.0,4.0,N,Great Ides,I truly would not get the mag lite with out th...,2000-06-10,410
1,51368106,R2JS2KYS2ES3B6,B00002N9ER,805377466,Maglite Black Universal Mounting Brackets for ...,Tools,1.0,24.0,10.0,34.0,N,Not worth the money at all,Both my wife and my electrician managed to bre...,2000-07-15,410
2,52720391,R1C5GE9X3VUSAC,B00002N9ER,805377466,Maglite Black Universal Mounting Brackets for ...,Tools,4.0,41.0,1.0,42.0,N,Keeps my Flashlight handy...,I have a set of these mounting brackets bolted...,2000-08-07,410
3,50471937,R1QUFA6PKKHJ16,B00002N9ER,805377466,Maglite Black Universal Mounting Brackets for ...,Tools,2.0,8.0,4.0,12.0,Y,Cheap,Poor quality plastic brackets .Not up to mag l...,2001-10-04,410
4,52457808,RJV9CGNM0U6W8,B00002N9ER,805377466,Maglite Black Universal Mounting Brackets for ...,Tools,5.0,23.0,2.0,25.0,N,Great for a Van or a Garage,"I have used these before, but the price here ...",2003-01-20,410


In [24]:

df.to_csv('new_df.csv')